## Iteration 3

In the third iteration, it was decided to strengthen the solution by adding the following features:
- Before sending the actual question, I added a few sample messages with responses to indicate that it is a few-shot prompt (to eliminate potential formatting issues).
- Added a framework for evaluation. First, it was decided to add a metric for failed JSON responses. Second, it was decided to add an LLM as a judge framework, which has its own criteria for checking whether the task meets the desired acceptance criteria (if the task contains a numbered list in the solution field, if the answer is formatted correctly, if the proper task type is returned, and so on). It was decided to implement it as an additional agent.

In [18]:
import json
import jsonlines
import pandas as pd
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

In [19]:
pd.set_option('display.max_colwidth', None)

In [20]:
load_dotenv()

True

In [21]:
with jsonlines.open('dataset.jsonl') as reader:
    dataset = list(reader)

dataset[0]

{'topic': 'Арифметичні дії',
 'type': 'task',
 'description': 'Обчисліть значення виразу: $100 - (15 + 5) \\times 4$.',
 'solution': '1. Виконайте дію в дужках: $15 + 5 = 20$. 2. Виконайте множення отриманого результату на 4: $20 \\times 4 = 80$. 3. Виконайте віднімання від 100: $100 - 80 = 20$.',
 'answer': '20'}

In [ ]:
generator_model = ChatOpenAI(model="gpt-4o", temperature=0.7)
judge_model = ChatOpenAI(model="gpt-4o", temperature=0.1)

In [23]:
class MathTutor:
    def __init__(self, model):
        self.model = model
        self.type_map = {'task': 'Задача', 'quiz': 'Тест'}
        self.system_prompt = """Ти - вчитель математики. Створи математичний контент на задану тему відповідно до вказаного типу.
Типи контенту:
1. Задача: 
   - description: умова задачі з одним питанням
   - solution: **нумерований** список кроків розв'язання
   - answer: число або рівняння
2. Тест:
   - description: умова задачі з чотирма варіантами відповіді (A, B, C, D)
   - solution: **нумерований** список кроків для знаходження відповіді
   - answer: одна латинська літера (A, B, C або D)
Відповідь надай у JSON форматі з трьома аттрибутами типу string: description, solution, answer. 
Не обгортай JSON у додаткові спецсимволи."""
        self.task_example = {
            'description': 'Обчисліть площу круга з радіусом $r = 4$. Прийміть $\\pi \\approx 3.14$.',
            'solution': '1. Згадайте формулу площі круга: $S = \\pi r^2$. 2. Підставте значення радіуса та числа $\\pi$: $S \\approx 3.14 \\cdot 4^2$. 3. Піднесіть радіус до квадрата: $4^2 = 16$. 4. Виконайте множення: $3.14 \\cdot 16 = 50.24$.',
            'answer': '50.24'
        }
        self.quiz_example = {
            'description': 'Знайдіть корінь рівняння $2x - 5 = 7$. \n A) 1 \n B) 6 \n C) 12 \n D) 2',
            'solution': '1. Перенесіть вільний член (-5) у праву частину рівняння, змінивши знак на протилежний: $2x = 7 + 5$. 2. Обчисліть суму в правій частині: $2x = 12$. 3. Поділіть обидві частини рівняння на коефіцієнт при змінній (на 2): $x = 12 / 2$. 4. Отримайте $x = 6$ і оберіть відповідний варіант B.',
            'answer': 'B'
        }

    def generate(self, topic, content_type):
        task_example_json = json.dumps(self.task_example, ensure_ascii=False)
        quiz_example_json = json.dumps(self.quiz_example, ensure_ascii=False)
        messages = [
            SystemMessage(content=self.system_prompt),
            HumanMessage(content="Тема: Площа круга, Тип: Задача"),
            AIMessage(content=task_example_json),
            HumanMessage(content="Тема: Лінійні рівняння, Тип: Тест"),
            AIMessage(content=quiz_example_json),
            HumanMessage(content=f"Тема: {topic}, Тип: {self.type_map[content_type]}")
        ]
        response = self.model.invoke(messages)
        return response.content

In [ ]:
class AcceptanceJudge:
    def __init__(self, model):
        self.model = model
        self.system_prompt = """Ти - експерт з оцінювання математичного контенту. Оціни наданий контент за критеріями.

Критерії:
1. has_numbered_solution: чи містить поле solution нумерований список кроків (1. ... 2. ... тощо)
2. has_valid_answer_format: 
   - для типу "task": відповідь має бути числом або рівнянням
   - для типу "quiz": відповідь має бути однією латинською літерою (A, B, C або D)
3. has_valid_description_format:
   - для типу "task": опис має містити умову задачі з питанням
   - для типу "quiz": опис має містити чотири варіанти відповіді (A, B, C, D)

Відповідь надай у JSON форматі з трьома булевими полями: has_numbered_solution, has_valid_answer_format, has_valid_description_format. 
Не обгортай JSON у додаткові спецсимволи."""
        self.task_example = {
            'description': 'Обчисліть площу круга з радіусом $r = 4$. Прийміть $\\pi \\approx 3.14$.',
            'solution': '1. Згадайте формулу площі круга: $S = \\pi r^2$. 2. Підставте значення радіуса та числа $\\pi$: $S \\approx 3.14 \\cdot 4^2$. 3. Піднесіть радіус до квадрата: $4^2 = 16$. 4. Виконайте множення: $3.14 \\cdot 16 = 50.24$.',
            'answer': '50.24'
        }
        self.quiz_example = {
            'description': 'Знайдіть корінь рівняння $2x - 5 = 7$. \n A) 1 \n B) 6 \n C) 12 \n D) 2',
            'solution': '1. Перенесіть вільний член (-5) у праву частину рівняння, змінивши знак на протилежний: $2x = 7 + 5$. 2. Обчисліть суму в правій частині: $2x = 12$. 3. Поділіть обидві частини рівняння на коефіцієнт при змінній (на 2): $x = 12 / 2$. 4. Отримайте $x = 6$ і оберіть відповідний варіант B.',
            'answer': 'B'
        }

    def evaluate(self, content, content_type):
        content_json = json.dumps(content, ensure_ascii=False)
        task_example_json = json.dumps(self.task_example, ensure_ascii=False)
        quiz_example_json = json.dumps(self.quiz_example, ensure_ascii=False)
        messages = [
            SystemMessage(content=self.system_prompt),
            HumanMessage(content=f"Тип контенту: task\n\nКонтент:\n{task_example_json}"),
            AIMessage(content='{"has_numbered_solution": true, "has_valid_answer_format": true, "has_valid_description_format": true}'),
            HumanMessage(content=f"Тип контенту: quiz\n\nКонтент:\n{quiz_example_json}"),
            AIMessage(content='{"has_numbered_solution": true, "has_valid_answer_format": true, "has_valid_description_format": true}'),
            HumanMessage(content=f"Тип контенту: {content_type}\n\nКонтент:\n{content_json}")
        ]
        response = self.model.invoke(messages)
        return response.content

In [ ]:
generator = MathTutor(generator_model)
judge = AcceptanceJudge(judge_model)

results = []

for item in dataset[:10]:

    generated_content = generator.generate(item['topic'], item['type'])
    
    try:
        parsed = json.loads(generated_content)
        is_valid_json = True
        generated_description = parsed.get('description', '')
        generated_solution = parsed.get('solution', '')
        generated_answer = parsed.get('answer', '')
    except json.JSONDecodeError:
        is_valid_json = False
        parsed = {}
        generated_description = ''
        generated_solution = ''
        generated_answer = ''

    if is_valid_json:
        judge_response = judge.evaluate(parsed, item['type'])
        try:
            judge_parsed = json.loads(judge_response)
            judge_valid_json = True
            has_numbered_solution = judge_parsed.get('has_numbered_solution', False)
            has_valid_answer_format = judge_parsed.get('has_valid_answer_format', False)
            has_valid_description_format = judge_parsed.get('has_valid_description_format', False)
        except json.JSONDecodeError:
            judge_valid_json = False
            has_numbered_solution = False
            has_valid_answer_format = False
            has_valid_description_format = False
    else:
        judge_response = ''
        judge_valid_json = False
        has_numbered_solution = False
        has_valid_answer_format = False
        has_valid_description_format = False
    
    display(pd.DataFrame([
        {'field': 'topic', 'value': item['topic']},
        {'field': 'type', 'value': item['type']},
        {'field': 'sample_description', 'value': item['description']},
        {'field': 'sample_solution', 'value': item['solution']},
        {'field': 'sample_answer', 'value': item['answer']},
        {'field': 'generated_content', 'value': generated_content},
        {'field': 'is_valid_json', 'value': is_valid_json},
        {'field': 'generated_description', 'value': generated_description},
        {'field': 'generated_solution', 'value': generated_solution},
        {'field': 'generated_answer', 'value': generated_answer},
        {'field': 'judge_response', 'value': judge_response},
        {'field': 'judge_valid_json', 'value': judge_valid_json},
        {'field': 'has_numbered_solution', 'value': has_numbered_solution},
        {'field': 'has_valid_answer_format', 'value': has_valid_answer_format},
        {'field': 'has_valid_description_format', 'value': has_valid_description_format},
    ]))

    results.append({
        'topic': item['topic'],
        'type': item['type'],
        'is_valid_json': is_valid_json,
        'judge_valid_json': judge_valid_json,
        'has_numbered_solution': has_numbered_solution,
        'has_valid_answer_format': has_valid_answer_format,
        'has_valid_description_format': has_valid_description_format,
    })

results = pd.DataFrame(results)

,field,value
0,topic,Арифметичні дії
1,type,task
2,sample_description,Обчисліть значення виразу: $100 - (15 + 5) \times 4$.
3,sample_solution,1. Виконайте дію в дужках: $15 + 5 = 20$. 2. Виконайте множення отриманого результату на 4: $20 \times 4 = 80$. 3. Виконайте віднімання від 100: $100 - 80 = 20$.
4,sample_answer,20
5,generated_content,"{""description"": ""На фермі є 15 корів, 23 курки та 12 свиней. Скільки всього тварин на фермі?"", ""solution"": ""1. Визначте кількість корів на фермі: 15. 2. Визначте кількість курок на фермі: 23. 3. Визначте кількість свиней на фермі: 12. 4. Складіть кількість усіх тварин: 15 + 23 + 12. 5. Обчисліть суму: 15 + 23 = 38, 38 + 12 = 50."", ""answer"": ""50""}"
6,is_valid_json,True
7,generated_description,"На фермі є 15 корів, 23 курки та 12 свиней. Скільки всього тварин на фермі?"
8,generated_solution,"1. Визначте кількість корів на фермі: 15. 2. Визначте кількість курок на фермі: 23. 3. Визначте кількість свиней на фермі: 12. 4. Складіть кількість усіх тварин: 15 + 23 + 12. 5. Обчисліть суму: 15 + 23 = 38, 38 + 12 = 50."
9,generated_answer,50


,field,value
0,topic,Геометрія
1,type,quiz
2,sample_description,Чому дорівнює сума внутрішніх кутів довільного трикутника? \n A) $90^\circ$ \n B) $180^\circ$ \n C) $270^\circ$ \n D) $360^\circ$
3,sample_solution,"1. Згадайте фундаментальну теорему геометрії про суму кутів трикутника. 2. Теорема стверджує, що сума кутів завжди дорівнює $180^\circ$. 3. Перевірте варіанти: A ($90^\circ$) — замало, C і D — забагато для трикутника. 4. Оберіть варіант B."
4,sample_answer,B
5,generated_content,"{""description"": ""Який з наведених чотирикутників є паралелограмом, якщо його діагоналі перетинаються і точка перетину ділить їх навпіл? \n A) Квадрат \n B) Прямокутник \n C) Ромб \n D) Усі зазначені"", ""solution"": ""1. Згадайте властивість паралелограма: діагоналі перетинаються в точці, яка ділить їх навпіл. 2. Визначте, які з наведених фігур мають цю властивість. 3. Квадрат є приватним випадком прямокутника і ромба, і він є паралелограмом. 4. Прямокутник також є паралелограмом, оскільки його діагоналі діляться навпіл. 5. Ромб є паралелограмом, тому що його діагоналі перетинаються під прямим кутом і діляться навпіл. 6. Оскільки всі три фігури задовольняють умову, правильна відповідь D."", ""answer"": ""D""}"
6,is_valid_json,True
7,generated_description,"Який з наведених чотирикутників є паралелограмом, якщо його діагоналі перетинаються і точка перетину ділить їх навпіл? \n A) Квадрат \n B) Прямокутник \n C) Ромб \n D) Усі зазначені"
8,generated_solution,"1. Згадайте властивість паралелограма: діагоналі перетинаються в точці, яка ділить їх навпіл. 2. Визначте, які з наведених фігур мають цю властивість. 3. Квадрат є приватним випадком прямокутника і ромба, і він є паралелограмом. 4. Прямокутник також є паралелограмом, оскільки його діагоналі діляться навпіл. 5. Ромб є паралелограмом, тому що його діагоналі перетинаються під прямим кутом і діляться навпіл. 6. Оскільки всі три фігури задовольняють умову, правильна відповідь D."
9,generated_answer,D


,field,value
0,topic,Спрощення виразів
1,type,task
2,sample_description,"Спростіть вираз, використовуючи формулу різниці квадратів: $(x - 3)(x + 3)$."
3,sample_solution,"1. Згадайте формулу різниці квадратів: $(a - b)(a + b) = a^2 - b^2$. 2. Підставте значення з виразу, де $a = x$ і $b = 3$. 3. Обчисліть квадрати: $x^2 - 3^2 = x^2 - 9$."
4,sample_answer,$x^2 - 9$
5,generated_content,"{""description"": ""Спростіть вираз $3(x + 2) - 4(x - 5)$. Яка його спрощена форма?"", ""solution"": ""1. Розкрийте дужки: $3(x + 2) = 3x + 6$ та $-4(x - 5) = -4x + 20$. 2. Запишіть вираз без дужок: $3x + 6 - 4x + 20$. 3. Згрупуйте подібні члени: $(3x - 4x) + (6 + 20)$. 4. Обчисліть: $-x + 26$."", ""answer"": ""-x + 26""}"
6,is_valid_json,True
7,generated_description,Спростіть вираз $3(x + 2) - 4(x - 5)$. Яка його спрощена форма?
8,generated_solution,1. Розкрийте дужки: $3(x + 2) = 3x + 6$ та $-4(x - 5) = -4x + 20$. 2. Запишіть вираз без дужок: $3x + 6 - 4x + 20$. 3. Згрупуйте подібні члени: $(3x - 4x) + (6 + 20)$. 4. Обчисліть: $-x + 26$.
9,generated_answer,-x + 26


,field,value
0,topic,Нерівності
1,type,quiz
2,sample_description,Який проміжок є розв'язком нерівності $x > 5$? \n A) $(-\infty; 5)$ \n B) $[5; +\infty)$ \n C) $(5; +\infty)$ \n D) $(-\infty; 5]$
3,sample_solution,"1. Проаналізуйте знак нерівності: $>$ (строго більше) означає, що число 5 не входить у множину розв'язків, тому дужка біля 5 має бути круглою. 2. Визначте напрямок: $x$ більше 5, отже проміжок йде вправо до $+\infty$. 3. Порівняйте з варіантами: B має квадратну дужку, A і D йдуть у неправильному напрямку. 4. Оберіть варіант C, який відповідає умовам."
4,sample_answer,C
5,generated_content,"{""description"": ""Яке з наступних чисел є розв'язком нерівності $3x + 4 < 10$? \n A) 0 \n B) 2 \n C) 3 \n D) 4"", ""solution"": ""1. Відніміть 4 з обох частин нерівності: $3x < 10 - 4$. 2. Обчисліть праву частину: $3x < 6$. 3. Поділіть обидві частини нерівності на 3: $x < 2$. 4. Перевірте кожен варіант: \n - A) 0: $0 < 2$ (вірно) \n - B) 2: $2 < 2$ (невірно) \n - C) 3: $3 < 2$ (невірно) \n - D) 4: $4 < 2$ (невірно) \n Оберіть варіант A."", ""answer"": ""A""}"
6,is_valid_json,True
7,generated_description,Яке з наступних чисел є розв'язком нерівності $3x + 4 < 10$? \n A) 0 \n B) 2 \n C) 3 \n D) 4
8,generated_solution,1. Відніміть 4 з обох частин нерівності: $3x < 10 - 4$. 2. Обчисліть праву частину: $3x < 6$. 3. Поділіть обидві частини нерівності на 3: $x < 2$. 4. Перевірте кожен варіант: \n - A) 0: $0 < 2$ (вірно) \n - B) 2: $2 < 2$ (невірно) \n - C) 3: $3 < 2$ (невірно) \n - D) 4: $4 < 2$ (невірно) \n Оберіть варіант A.
9,generated_answer,A


,field,value
0,topic,Системи рівнянь
1,type,task
2,sample_description,Розв'яжіть систему рівнянь: $\begin{cases} x + y = 10 \\ x - y = 4 \end{cases}$
3,sample_solution,"1. Застосуйте метод додавання рівнянь: $(x + y) + (x - y) = 10 + 4$. 2. Спростіть отримане рівняння: $2x = 14$. 3. Знайдіть $x$, поділивши обидві частини на 2: $x = 7$. 4. Підставте значення $x$ у перше рівняння ($7 + y = 10$) та знайдіть $y$: $y = 3$."
4,sample_answer,"$x = 7, y = 3$"
5,generated_content,"{""description"": ""Розв'яжіть систему рівнянь: \\( \\begin{cases} 2x + 3y = 12 \\ 4x - y = 5 \\end{cases} \\)"", ""solution"": ""1. Виберіть рівняння для вираження однієї змінної через іншу. Виберемо друге рівняння: $4x - y = 5$. 2. Виразіть $y$ через $x$: $y = 4x - 5$. 3. Підставте вираз для $y$ у перше рівняння: $2x + 3(4x - 5) = 12$. 4. Розкрийте дужки: $2x + 12x - 15 = 12$. 5. Спрощення: $14x - 15 = 12$. 6. Додайте 15 до обох частин: $14x = 27$. 7. Поділіть обидві частини на 14: $x = \\frac{27}{14}$. 8. Підставте значення $x$ у вираз для $y$: $y = 4(\\frac{27}{14}) - 5$. 9. Обчисліть: $y = \\frac{108}{14} - 5 = \\frac{108}{14} - \\frac{70}{14} = \\frac{38}{14} = \\frac{19}{7}$."", ""answer"": ""x = \\frac{27}{14}, y = \\frac{19}{7}""}"
6,is_valid_json,True
7,generated_description,Розв'яжіть систему рівнянь: \( \begin{cases} 2x + 3y = 12 \ 4x - y = 5 \end{cases} \)
8,generated_solution,1. Виберіть рівняння для вираження однієї змінної через іншу. Виберемо друге рівняння: $4x - y = 5$. 2. Виразіть $y$ через $x$: $y = 4x - 5$. 3. Підставте вираз для $y$ у перше рівняння: $2x + 3(4x - 5) = 12$. 4. Розкрийте дужки: $2x + 12x - 15 = 12$. 5. Спрощення: $14x - 15 = 12$. 6. Додайте 15 до обох частин: $14x = 27$. 7. Поділіть обидві частини на 14: $x = \frac{27}{14}$. 8. Підставте значення $x$ у вираз для $y$: $y = 4(\frac{27}{14}) - 5$. 9. Обчисліть: $y = \frac{108}{14} - 5 = \frac{108}{14} - \frac{70}{14} = \frac{38}{14} = \frac{19}{7}$.
9,generated_answer,"x = \frac{27}{14}, y = \frac{19}{7}"


,field,value
0,topic,Квадратні корені
1,type,quiz
2,sample_description,Обчисліть значення виразу $\sqrt{144}$. \n A) 10 \n B) 11 \n C) 12 \n D) 14
3,sample_solution,"1. Визначте, що потрібно знайти число, яке при множенні саме на себе дає 144. 2. Перевірте варіанти відповідей піднесенням до квадрату. 3. $10^2=100$, $11^2=121$, $12^2=144$. 4. Оскільки $12^2=144$, оберіть варіант C."
4,sample_answer,C
5,generated_content,"{""description"": ""Знайдіть значення виразу $\\sqrt{49} + \\sqrt{16}$. \n A) 25 \n B) 11 \n C) 10 \n D) 9"", ""solution"": ""1. Обчисліть квадратний корінь з 49: $\\sqrt{49} = 7$. 2. Обчисліть квадратний корінь з 16: $\\sqrt{16} = 4$. 3. Додайте отримані значення: $7 + 4 = 11$. 4. Оберіть відповідний варіант B."", ""answer"": ""B""}"
6,is_valid_json,True
7,generated_description,Знайдіть значення виразу $\sqrt{49} + \sqrt{16}$. \n A) 25 \n B) 11 \n C) 10 \n D) 9
8,generated_solution,1. Обчисліть квадратний корінь з 49: $\sqrt{49} = 7$. 2. Обчисліть квадратний корінь з 16: $\sqrt{16} = 4$. 3. Додайте отримані значення: $7 + 4 = 11$. 4. Оберіть відповідний варіант B.
9,generated_answer,B


,field,value
0,topic,Арифметична прогресія
1,type,task
2,sample_description,"Дано арифметичну прогресію, де $a_1 = 5$ і різниця $d = 3$. Знайдіть п'ятий член прогресії ($a_5$)."
3,sample_solution,"1. Запишіть формулу $n$-го члена арифметичної прогресії: $a_n = a_1 + d(n - 1)$. 2. Підставте відомі значення ($a_1=5$, $d=3$, $n=5$) у формулу. 3. Виконайте обчислення в дужках: $5 - 1 = 4$. 4. Завершіть обчислення: $5 + 3 \times 4 = 5 + 12 = 17$."
4,sample_answer,17
5,generated_content,"{""description"": ""Знайдіть 10-й член арифметичної прогресії, якщо перший член дорівнює 3, а різниця дорівнює 2."", ""solution"": ""1. Згадайте формулу n-го члена арифметичної прогресії: $a_n = a_1 + (n-1)d$. 2. Підставте значення першого члена ($a_1 = 3$), різниці ($d = 2$) та номеру члена ($n = 10$) у формулу: $a_{10} = 3 + (10-1) \\cdot 2$. 3. Обчисліть вираз у дужках: $10 - 1 = 9$. 4. Помножте різницю на результат з попереднього кроку: $9 \\cdot 2 = 18$. 5. Додайте результат до першого члена: $3 + 18 = 21$."", ""answer"": ""21""}"
6,is_valid_json,True
7,generated_description,"Знайдіть 10-й член арифметичної прогресії, якщо перший член дорівнює 3, а різниця дорівнює 2."
8,generated_solution,"1. Згадайте формулу n-го члена арифметичної прогресії: $a_n = a_1 + (n-1)d$. 2. Підставте значення першого члена ($a_1 = 3$), різниці ($d = 2$) та номеру члена ($n = 10$) у формулу: $a_{10} = 3 + (10-1) \cdot 2$. 3. Обчисліть вираз у дужках: $10 - 1 = 9$. 4. Помножте різницю на результат з попереднього кроку: $9 \cdot 2 = 18$. 5. Додайте результат до першого члена: $3 + 18 = 21$."
9,generated_answer,21


,field,value
0,topic,Тригонометрія
1,type,quiz
2,sample_description,Чому дорівнює значення $\sin 30^\circ$? \n A) $1$ \n B) $0.5$ \n C) $\frac{\sqrt{2}}{2}$ \n D) $\frac{\sqrt{3}}{2}$
3,sample_solution,"1. Зверніться до таблиці значень тригонометричних функцій. 2. Знайдіть значення для синуса кута $30^\circ$. 3. Табличне значення $\sin 30^\circ = 1/2$ або $0.5$. 4. Порівняйте з варіантами: C — це $45^\circ$, D — це $60^\circ$. 5. Оберіть варіант B."
4,sample_answer,B
5,generated_content,"{""description"": ""Знайдіть значення $\\sin(30^\\circ)$. \n A) 0.5 \n B) \\sqrt{3}/2 \n C) 1 \n D) 0"", ""solution"": ""1. Згадайте основні значення тригонометричних функцій для стандартних кутів. 2. Відомо, що $\\sin(30^\\circ) = 1/2$. 3. Перевірте відповідність варіантів відповіді з обчисленим значенням: $1/2 = 0.5$. 4. Виберіть варіант A, оскільки він відповідає значенню 0.5."", ""answer"": ""A""}"
6,is_valid_json,True
7,generated_description,Знайдіть значення $\sin(30^\circ)$. \n A) 0.5 \n B) \sqrt{3}/2 \n C) 1 \n D) 0
8,generated_solution,"1. Згадайте основні значення тригонометричних функцій для стандартних кутів. 2. Відомо, що $\sin(30^\circ) = 1/2$. 3. Перевірте відповідність варіантів відповіді з обчисленим значенням: $1/2 = 0.5$. 4. Виберіть варіант A, оскільки він відповідає значенню 0.5."
9,generated_answer,A


,field,value
0,topic,Логарифми
1,type,task
2,sample_description,Обчисліть значення логарифма: $\log_2 32$.
3,sample_solution,"1. Визначте означення логарифма: це степінь, до якого треба піднести основу (2), щоб отримати аргумент (32). 2. Перевірте степені двійки: $2^1=2$, $2^2=4$, $2^3=8$, $2^4=16$, $2^5=32$. 3. Оскільки $2^5 = 32$, значення логарифма дорівнює 5."
4,sample_answer,5
5,generated_content,"{""description"": ""Обчисліть значення виразу $\\log_2 32$."", ""solution"": ""1. Згадайте властивість логарифма: $\\log_b a = c$ означає, що $b^c = a$. 2. Підставте значення з умови: $2^c = 32$. 3. Розкладіть 32 як степінь числа 2: $32 = 2^5$. 4. Зрівняйте показники степенів: $c = 5$."", ""answer"": ""5""}"
6,is_valid_json,True
7,generated_description,Обчисліть значення виразу $\log_2 32$.
8,generated_solution,"1. Згадайте властивість логарифма: $\log_b a = c$ означає, що $b^c = a$. 2. Підставте значення з умови: $2^c = 32$. 3. Розкладіть 32 як степінь числа 2: $32 = 2^5$. 4. Зрівняйте показники степенів: $c = 5$."
9,generated_answer,5


,field,value
0,topic,Функції
1,type,quiz
2,sample_description,Яка з наведених функцій є лінійною? \n A) $y = x^2$ \n B) $y = \frac{1}{x}$ \n C) $y = 2x + 5$ \n D) $y = \sqrt{x}$
3,sample_solution,"1. Згадайте означення лінійної функції: $y = kx + b$, де $x$ у першому степені. 2. Проаналізуйте варіант A: $x^2$ — квадратична функція. 3. Проаналізуйте варіант B: $1/x$ — обернена пропорційність. 4. Проаналізуйте варіант C: відповідає вигляду $kx + b$ ($k=2, b=5$). 5. Оберіть варіант C."
4,sample_answer,C
5,generated_content,"{""description"": ""Знайдіть значення функції $f(x) = 3x + 7$ при $x = 2$. \n A) 10 \n B) 13 \n C) 16 \n D) 19"", ""solution"": ""1. Підставте значення $x = 2$ у функцію: $f(2) = 3(2) + 7$. 2. Виконайте множення: $3(2) = 6$. 3. Додайте 7: $6 + 7 = 13$. 4. Отримане значення дорівнює 13, тому оберіть варіант B."", ""answer"": ""B""}"
6,is_valid_json,True
7,generated_description,Знайдіть значення функції $f(x) = 3x + 7$ при $x = 2$. \n A) 10 \n B) 13 \n C) 16 \n D) 19
8,generated_solution,"1. Підставте значення $x = 2$ у функцію: $f(2) = 3(2) + 7$. 2. Виконайте множення: $3(2) = 6$. 3. Додайте 7: $6 + 7 = 13$. 4. Отримане значення дорівнює 13, тому оберіть варіант B."
9,generated_answer,B


In [ ]:
display(results)

bool_columns = ['is_valid_json', 'judge_valid_json', 'has_numbered_solution', 'has_valid_answer_format', 'has_valid_description_format']
failures = results[~results[bool_columns].all(axis=1)]

print(f"Failures: {len(failures)} / {len(results)}")
if failures:
    display(failures)

,topic,type,is_valid_json,judge_valid_json,has_numbered_solution,has_valid_answer_format,has_valid_description_format
0,Арифметичні дії,task,True,True,True,True,True
1,Геометрія,quiz,True,True,True,True,True
2,Спрощення виразів,task,True,True,True,True,True
3,Нерівності,quiz,True,True,True,True,True
4,Системи рівнянь,task,True,True,True,True,True
5,Квадратні корені,quiz,True,True,True,True,True
6,Арифметична прогресія,task,True,True,True,True,True
7,Тригонометрія,quiz,True,True,True,True,True
8,Логарифми,task,True,True,True,True,True
9,Функції,quiz,True,True,True,True,True


Failures: 0 / 10


,topic,type,is_valid_json,judge_valid_json,has_numbered_solution,has_valid_answer_format,has_valid_description_format
